<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/NN_1_10_Relu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models
from pgmpy.estimators import HillClimbSearch, BicScore, AICScore, MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.metrics import accuracy_score
from scipy.stats import entropy
import os
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------------------------------------

# Bayesian Network Data Generation 500, 1000, 1500, ..., 10000 Samples (dense)

In [3]:
#np.random.seed(1)

# Define the mappings for IR, EI, SP
ir_map = {0: 'low', 1: 'medium', 2: 'high'}
ei_map = {0: 'poor', 1: 'average', 2: 'good'}
sp_map = {0: 'decrease', 1: 'stable', 2: 'increase'}

# Define the dense Bayesian Network
dense_model = BayesianNetwork([('IR', 'EI'), ('EI', 'SP'), ('IR', 'SP')])

# Function to generate CPDs
def generate_cpds():
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()  # Normalize to make it a valid probability distribution

    ei_given_ir_probs = np.random.rand(3, 3)
    ei_given_ir_probs /= ei_given_ir_probs.sum(axis=0, keepdims=True)

    sp_probs = np.random.rand(3, 3, 3)
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    sp_probs_reshaped = sp_probs.reshape(3, -1)

    return ir_probs, ei_given_ir_probs, sp_probs_reshaped

# Save probabilities in a single CSV file
def save_probabilities(ir_probs, ei_probs, sp_probs, filename):
    # Create a DataFrame for IR probabilities
    ir_df = pd.DataFrame({
        'IR_State': ['low', 'medium', 'high'],
        'IR_Prob': ir_probs
    })

    # Create a DataFrame for EI given IR probabilities
    ei_df = pd.DataFrame(ei_probs, columns=['EI_given_IR_low', 'EI_given_IR_medium', 'EI_given_IR_high'])
    ei_df['EI_State'] = ['poor', 'average', 'good']

    # Create a DataFrame for SP given IR and EI probabilities
    sp_df = pd.DataFrame(sp_probs, columns=[
        'SP_given_IR_low_EI_poor', 'SP_given_IR_low_EI_average', 'SP_given_IR_low_EI_good',
        'SP_given_IR_medium_EI_poor', 'SP_given_IR_medium_EI_average', 'SP_given_IR_medium_EI_good',
        'SP_given_IR_high_EI_poor', 'SP_given_IR_high_EI_average', 'SP_given_IR_high_EI_good'
    ])
    sp_df['SP_State'] = ['decrease', 'stable', 'increase']

    # Combine all data into a single DataFrame
    combined_df = pd.concat([ir_df, ei_df, sp_df], axis=1)

    # Save the combined DataFrame as a single CSV file
    combined_df.to_csv(filename, index=False)

# Save outcomes in a CSV file
def save_outcomes(data_dense, filename):
    data_dense['IR'] = data_dense['IR'].map(ir_map)
    data_dense['EI'] = data_dense['EI'].map(ei_map)
    data_dense['SP'] = data_dense['SP'].map(sp_map)
    data_dense.to_csv(filename, index=False)

# Generate datasets for different sample sizes for the dense model
sample_sizes = range(500, 10500, 500)
for size in sample_sizes:
    # Generate the CPDs
    ir_probs, ei_given_ir_probs, sp_probs_reshaped = generate_cpds()

    # Define CPDs for the dense model
    cpd_ir = TabularCPD(variable='IR', variable_card=3, values=[[ir_probs[0]], [ir_probs[1]], [ir_probs[2]]])
    cpd_ei_dense = TabularCPD(variable='EI', variable_card=3,
                              values=ei_given_ir_probs,
                              evidence=['IR'], evidence_card=[3])
    cpd_sp_dense = TabularCPD(variable='SP', variable_card=3,
                              values=sp_probs_reshaped,
                              evidence=['IR', 'EI'], evidence_card=[3, 3])

    dense_model.add_cpds(cpd_ir, cpd_ei_dense, cpd_sp_dense)

    # Check if the model is valid
    assert dense_model.check_model()

    # Generate samples
    sampler_dense = BayesianModelSampling(dense_model)
    data_dense = sampler_dense.forward_sample(size=size)

    # Save probabilities in one file
    save_probabilities(ir_probs, ei_given_ir_probs, sp_probs_reshaped, f'probabilities_dense_{size}.csv')

    # Save outcomes (low, medium, high) in another file
    save_outcomes(data_dense, f'outcomes_dense_{size}.csv')

# Notify the user that the process is done
print("Data generation and saving complete for the dense model!")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Data generation and saving complete for the dense model!


# Bayesian Network Data Generation 500, 1000, 1500, ..., 10000 Samples (sparse)

In [4]:
#np.random.seed(187)

# Define the mappings for IR, EI, SP
ir_map = {0: 'low', 1: 'medium', 2: 'high'}
ei_map = {0: 'poor', 1: 'average', 2: 'good'}
sp_map = {0: 'decrease', 1: 'stable', 2: 'increase'}

# Define the sparse Bayesian Network
sparse_model = BayesianNetwork([('IR', 'SP'), ('EI', 'SP')])

# Function to generate CPDs for the sparse model
def generate_cpds_sparse():
    # Generate probabilities for IR (unconditional)
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()  # Normalize to make it a valid probability distribution

    # Generate unconditional probabilities for EI (no dependency on IR)
    ei_probs = np.random.rand(3)
    ei_probs /= ei_probs.sum()

    # Generate conditional probabilities for SP given IR and EI
    sp_probs = np.random.rand(3, 3, 3)
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    sp_probs_reshaped = sp_probs.reshape(3, -1)

    return ir_probs, ei_probs, sp_probs_reshaped

# Save probabilities in a single CSV file
def save_probabilities_sparse(ir_probs, ei_probs, sp_probs, filename):
    # Create a DataFrame for IR probabilities
    ir_df = pd.DataFrame({
        'IR_State': ['low', 'medium', 'high'],
        'IR_Prob': ir_probs
    })

    # Create a DataFrame for EI probabilities (since it's not conditional on IR)
    ei_df = pd.DataFrame({
        'EI_State': ['poor', 'average', 'good'],
        'EI_Prob': ei_probs
    })

    # Create a DataFrame for SP given IR and EI probabilities
    sp_df = pd.DataFrame(sp_probs, columns=[
        'SP_given_IR_low_EI_poor', 'SP_given_IR_low_EI_average', 'SP_given_IR_low_EI_good',
        'SP_given_IR_medium_EI_poor', 'SP_given_IR_medium_EI_average', 'SP_given_IR_medium_EI_good',
        'SP_given_IR_high_EI_poor', 'SP_given_IR_high_EI_average', 'SP_given_IR_high_EI_good'
    ])
    sp_df['SP_State'] = ['decrease', 'stable', 'increase']

    # Combine all data into a single DataFrame
    combined_df = pd.concat([ir_df, ei_df, sp_df], axis=1)

    # Save the combined DataFrame as a single CSV file
    combined_df.to_csv(filename, index=False)

# Save outcomes in a CSV file
def save_outcomes_sparse(data_sparse, filename):
    data_sparse['IR'] = data_sparse['IR'].map(ir_map)
    data_sparse['EI'] = data_sparse['EI'].map(ei_map)
    data_sparse['SP'] = data_sparse['SP'].map(sp_map)
    data_sparse.to_csv(filename, index=False)

# Generate datasets for different sample sizes for the sparse model
sample_sizes = range(500, 10500, 500)
for size in sample_sizes:
    # Generate the CPDs
    ir_probs, ei_probs, sp_probs_reshaped = generate_cpds_sparse()

    # Define CPDs for the sparse model
    cpd_ir = TabularCPD(variable='IR', variable_card=3, values=[[ir_probs[0]], [ir_probs[1]], [ir_probs[2]]])
    cpd_ei_sparse = TabularCPD(variable='EI', variable_card=3, values=[[ei_probs[0]], [ei_probs[1]], [ei_probs[2]]])
    cpd_sp_sparse = TabularCPD(variable='SP', variable_card=3,
                               values=sp_probs_reshaped,
                               evidence=['IR', 'EI'], evidence_card=[3, 3])

    sparse_model.add_cpds(cpd_ir, cpd_ei_sparse, cpd_sp_sparse)

    # Check if the model is valid
    assert sparse_model.check_model()

    # Generate samples
    sampler_sparse = BayesianModelSampling(sparse_model)
    data_sparse = sampler_sparse.forward_sample(size=size)

    # Save probabilities in one file
    save_probabilities_sparse(ir_probs, ei_probs, sp_probs_reshaped, f'probabilities_sparse_{size}.csv')

    # Save outcomes (low, medium, high) in another file
    save_outcomes_sparse(data_sparse, f'outcomes_sparse_{size}.csv')

# Notify the user that the process is done
print("Data generation and saving complete for the sparse model!")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Data generation and saving complete for the sparse model!


In [ ]:
# Sample sizes to loop through
sample_sizes = range(500, 10500, 500)

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing sample size: {sample_size}")

    # Load the dense dataset for the current sample size
    dense_data_file = f'outcomes_dense_{sample_size}.csv'
    df_dense = pd.read_csv(dense_data_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df_dense['IR_encoded'] = df_dense['IR'].map(ir_map)
    df_dense['EI_encoded'] = df_dense['EI'].map(ei_map)
    df_dense['SP_encoded'] = df_dense['SP'].map(sp_map)

    # Define the Hill-Climb structure learning algorithm
    hc = HillClimbSearch(df_dense[['IR_encoded', 'EI_encoded', 'SP_encoded']])
    scoring_method = BicScore(df_dense[['IR_encoded', 'EI_encoded', 'SP_encoded']])

    # Estimate the best structure
    best_dag = hc.estimate(scoring_method=scoring_method)
    best_model = BayesianNetwork(best_dag.edges())

    # Check if all nodes are included in the learned structure
    nodes_in_structure = set(best_model.nodes())
    required_nodes = {'IR_encoded', 'EI_encoded', 'SP_encoded'}

    if not required_nodes.issubset(nodes_in_structure):
        print("\nNot all nodes are connected. Adding a dummy variable.")
        # Add a dummy variable to the dataset
        df_dense['dummy'] = 0

        # Re-estimate the structure with the dummy variable
        hc = HillClimbSearch(df_dense)
        scoring_method = BicScore(df_dense)
        best_dag = hc.estimate(scoring_method=scoring_method)
        best_model = BayesianNetwork(best_dag.edges())

    # Display the learned structure (edges of the Bayesian Network)
    print(f"\nLearned Structure (Edges) for {sample_size} samples:")
    print(best_model.edges())

    # Calculate and display the BIC score
    bic_score = scoring_method.score(best_model)
    print(f"\nBIC Score for {sample_size} samples: {bic_score:.4f}")

    # Learn the CPDs using Maximum Likelihood Estimation (MLE)
    best_model.fit(df_dense, estimator=MaximumLikelihoodEstimator)

    # Check if the model is valid after learning the parameters
    assert best_model.check_model()

    # Print the learned CPDs (Conditional Probability Distributions)
    for cpd in best_model.get_cpds():
        print("\nCPD of", cpd.variable)
        print(cpd)

    # Create an inference object for the best model
    inference = VariableElimination(best_model)

    # Placeholder to store predictions
    predicted_sp_labels = []

    # Loop through each row in the dense dataset to make predictions
    for index, row in df_dense.iterrows():
        # Prepare the evidence from the dataset (IR_encoded and EI_encoded)
        sample_input = {'IR_encoded': int(row['IR_encoded']), 'EI_encoded': int(row['EI_encoded'])}

        # Perform inference to predict the distribution for SP_encoded (Stock Price)
        predicted_sp_distribution = inference.query(variables=['SP_encoded'], evidence=sample_input)

        # Extract the most likely SP_encoded class
        predicted_sp_class = predicted_sp_distribution.values.argmax()
        sp_reverse_map = {0: 'decrease', 1: 'stable', 2: 'increase'}
        predicted_sp_label = sp_reverse_map[predicted_sp_class]

        # Store the predicted label
        predicted_sp_labels.append(predicted_sp_label)

    # Convert the list of predicted labels into a DataFrame for easier comparison
    predicted_results_df = pd.DataFrame({
        'IR': df_dense['IR'],  # Original IR column
        'EI': df_dense['EI'],  # Original EI column
        'Predicted_SP': predicted_sp_labels  # Predicted SP column
    })

    # Add the actual SP values for comparison
    predicted_results_df['Actual_SP'] = df_dense['SP']

    # Calculate accuracy of predictions
    accuracy = accuracy_score(predicted_results_df['Actual_SP'], predicted_results_df['Predicted_SP'])
    print(f"\nPrediction Accuracy for {sample_size} samples: {accuracy:.4f}")

    # Display the first few rows of predictions
    print(f"\nPredicted Results for Dense Data (First 10 rows) for {sample_size} samples:")
    print(predicted_results_df.head(10))

# Notify the user that the process is done
print("\nProcessing complete for all sample sizes.")


Processing sample size: 500


  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 500 samples:
[('IR_encoded', 'SP_encoded'), ('IR_encoded', 'EI_encoded'), ('EI_encoded', 'SP_encoded')]

BIC Score for 500 samples: -1319.6222

CPD of IR_encoded
+---------------+-------+
| IR_encoded(0) | 0.352 |
+---------------+-------+
| IR_encoded(1) | 0.648 |
+---------------+-------+

CPD of SP_encoded
+---------------+---------------------+-----+----------------------+---------------------+
| EI_encoded    | EI_encoded(0)       | ... | EI_encoded(2)        | EI_encoded(2)       |
+---------------+---------------------+-----+----------------------+---------------------+
| IR_encoded    | IR_encoded(0)       | ... | IR_encoded(0)        | IR_encoded(1)       |
+---------------+---------------------+-----+----------------------+---------------------+
| SP_encoded(0) | 0.12727272727272726 | ... | 0.023809523809523808 | 0.6122448979591837  |
+---------------+---------------------+-----+----------------------+---------------------+
| SP_encoded(1) | 0.8

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 1000 samples:
[('IR_encoded', 'EI_encoded'), ('EI_encoded', 'SP_encoded')]

BIC Score for 1000 samples: -2711.4516

CPD of IR_encoded
+---------------+-------+
| IR_encoded(0) | 0.525 |
+---------------+-------+
| IR_encoded(1) | 0.057 |
+---------------+-------+
| IR_encoded(2) | 0.418 |
+---------------+-------+

CPD of EI_encoded
+---------------+---------------------+---------------------+---------------------+
| IR_encoded    | IR_encoded(0)       | IR_encoded(1)       | IR_encoded(2)       |
+---------------+---------------------+---------------------+---------------------+
| EI_encoded(0) | 0.11238095238095239 | 0.03508771929824561 | 0.34688995215311    |
+---------------+---------------------+---------------------+---------------------+
| EI_encoded(1) | 0.8552380952380952  | 0.5087719298245614  | 0.17703349282296652 |
+---------------+---------------------+---------------------+---------------------+
| EI_encoded(2) | 0.03238095238095238 | 0.4561

...


Learned Structure (Edges) for 9000 samples:
[('IR_encoded', 'EI_encoded'), ('SP_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

BIC Score for 9000 samples: -25678.4204

CPD of IR_encoded
+---------------+----------------------+----------------------+---------------------+
| SP_encoded    | SP_encoded(0)        | SP_encoded(1)        | SP_encoded(2)       |
+---------------+----------------------+----------------------+---------------------+
| IR_encoded(0) | 0.307740130556419    | 0.5086916742909423   | 0.45726837060702874 |
+---------------+----------------------+----------------------+---------------------+
| IR_encoded(1) | 0.6673919801056886   | 0.46996035376639217  | 0.4972044728434505  |
+---------------+----------------------+----------------------+---------------------+
| IR_encoded(2) | 0.024867889337892447 | 0.021347971942665446 | 0.04552715654952077 |
+---------------+----------------------+----------------------+---------------------+

CPD of EI_encoded
+----------

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 9500 samples:
[('IR_encoded', 'EI_encoded'), ('IR_encoded', 'SP_encoded'), ('SP_encoded', 'EI_encoded')]

BIC Score for 9500 samples: -28290.2908

CPD of IR_encoded
+---------------+----------+
| IR_encoded(0) | 0.196737 |
+---------------+----------+
| IR_encoded(1) | 0.319684 |
+---------------+----------+
| IR_encoded(2) | 0.483579 |
+---------------+----------+

CPD of EI_encoded
+---------------+----------------------+-----+---------------------+---------------------+
| IR_encoded    | IR_encoded(0)        | ... | IR_encoded(2)       | IR_encoded(2)       |
+---------------+----------------------+-----+---------------------+---------------------+
| SP_encoded    | SP_encoded(0)        | ... | SP_encoded(1)       | SP_encoded(2)       |
+---------------+----------------------+-----+---------------------+---------------------+
| EI_encoded(0) | 0.2945054945054945   | ... | 0.3809782608695652  | 0.42289239204934886 |
+---------------+-------------------

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 10000 samples:
[('IR_encoded', 'EI_encoded'), ('IR_encoded', 'SP_encoded'), ('SP_encoded', 'EI_encoded')]

BIC Score for 10000 samples: -29479.1864

CPD of IR_encoded
+---------------+--------+
| IR_encoded(0) | 0.5033 |
+---------------+--------+
| IR_encoded(1) | 0.2318 |
+---------------+--------+
| IR_encoded(2) | 0.2649 |
+---------------+--------+

CPD of EI_encoded
+---------------+---------------------+-----+---------------------+---------------------+
| IR_encoded    | IR_encoded(0)       | ... | IR_encoded(2)       | IR_encoded(2)       |
+---------------+---------------------+-----+---------------------+---------------------+
| SP_encoded    | SP_encoded(0)       | ... | SP_encoded(1)       | SP_encoded(2)       |
+---------------+---------------------+-----+---------------------+---------------------+
| EI_encoded(0) | 0.03593339176161262 | ... | 0.12480252764612954 | 0.4421052631578947  |
+---------------+---------------------+-----+---------

## AIC

In [ ]:
# Sample sizes to loop through
sample_sizes = range(500, 10500, 500)

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing sample size: {sample_size}")

    # Load the dense dataset for the current sample size
    dense_data_file = f'outcomes_dense_{sample_size}.csv'
    df_dense = pd.read_csv(dense_data_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df_dense['IR_encoded'] = df_dense['IR'].map(ir_map)
    df_dense['EI_encoded'] = df_dense['EI'].map(ei_map)
    df_dense['SP_encoded'] = df_dense['SP'].map(sp_map)

    # Define the Hill-Climb structure learning algorithm
    hc = HillClimbSearch(df_dense[['IR_encoded', 'EI_encoded', 'SP_encoded']])
    scoring_method = AICScore(df_dense[['IR_encoded', 'EI_encoded', 'SP_encoded']])  # Use AICScore instead of BicScore

    # Estimate the best structure
    best_dag = hc.estimate(scoring_method=scoring_method)
    best_model = BayesianNetwork(best_dag.edges())

    # Display the learned structure (edges of the Bayesian Network)
    print(f"\nLearned Structure (Edges) for {sample_size} samples (AIC):")
    print(best_model.edges())

    # Calculate and display the AIC score
    aic_score = scoring_method.score(best_model)
    print(f"\nAIC Score for {sample_size} samples: {aic_score:.4f}")

    # Learn the CPDs using Maximum Likelihood Estimation (MLE)
    best_model.fit(df_dense[['IR_encoded', 'EI_encoded', 'SP_encoded']], estimator=MaximumLikelihoodEstimator)

    # Check if the model is valid after learning the parameters
    assert best_model.check_model()

    # Print the learned CPDs (Conditional Probability Distributions)
    for cpd in best_model.get_cpds():
        print("\nCPD of", cpd.variable)
        print(cpd)

    # Create an inference object for the best model
    inference = VariableElimination(best_model)

    # Placeholder to store predictions
    predicted_sp_labels = []

    # Loop through each row in the dense dataset to make predictions
    for index, row in df_dense.iterrows():
        # Prepare the evidence from the dataset (IR_encoded and EI_encoded)
        sample_input = {'IR_encoded': int(row['IR_encoded']), 'EI_encoded': int(row['EI_encoded'])}

        # Perform inference to predict the distribution for SP_encoded (Stock Price)
        predicted_sp_distribution = inference.query(variables=['SP_encoded'], evidence=sample_input)

        # Extract the most likely SP_encoded class
        predicted_sp_class = predicted_sp_distribution.values.argmax()
        sp_reverse_map = {0: 'decrease', 1: 'stable', 2: 'increase'}
        predicted_sp_label = sp_reverse_map[predicted_sp_class]

        # Store the predicted label
        predicted_sp_labels.append(predicted_sp_label)

    # Convert the list of predicted labels into a DataFrame for easier comparison
    predicted_results_df = pd.DataFrame({
        'IR': df_dense['IR'],  # Original IR column
        'EI': df_dense['EI'],  # Original EI column
        'Predicted_SP': predicted_sp_labels  # Predicted SP column
    })

    # Add the actual SP values for comparison
    predicted_results_df['Actual_SP'] = df_dense['SP']

    # Calculate accuracy of predictions
    accuracy = accuracy_score(predicted_results_df['Actual_SP'], predicted_results_df['Predicted_SP'])
    print(f"\nPrediction Accuracy for {sample_size} samples (AIC): {accuracy:.4f}")

    # Display the first few rows of predictions
    print(f"\nPredicted Results for Dense Data (First 10 rows) for {sample_size} samples (AIC):")
    print(predicted_results_df.head(10))

    # Save the results if needed
    results_filename = f'predicted_results_aic_{sample_size}.csv'
    predicted_results_df.to_csv(results_filename, index=False)
    print(f"\nResults saved to {results_filename}")

# Notify the user that the process is done
print("\nProcessing complete for all sample sizes using AIC.")


Processing sample size: 500


  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 500 samples (AIC):
[('IR_encoded', 'EI_encoded'), ('SP_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

AIC Score for 500 samples: -1283.7981

CPD of IR_encoded
+---------------+---------------------+--------------------+---------------------+
| SP_encoded    | SP_encoded(0)       | SP_encoded(1)      | SP_encoded(2)       |
+---------------+---------------------+--------------------+---------------------+
| IR_encoded(0) | 0.17532467532467533 | 0.4550561797752809 | 0.40476190476190477 |
+---------------+---------------------+--------------------+---------------------+
| IR_encoded(1) | 0.8246753246753247  | 0.5449438202247191 | 0.5952380952380952  |
+---------------+---------------------+--------------------+---------------------+

CPD of EI_encoded
+---------------+---------------------+-----+---------------------+---------------+
| IR_encoded    | IR_encoded(0)       | ... | IR_encoded(1)       | IR_encoded(1) |
+---------------+----------------

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 1000 samples (AIC):
[('IR_encoded', 'EI_encoded'), ('IR_encoded', 'SP_encoded'), ('SP_encoded', 'EI_encoded')]

AIC Score for 1000 samples: -2669.6189

CPD of IR_encoded
+---------------+-------+
| IR_encoded(0) | 0.525 |
+---------------+-------+
| IR_encoded(1) | 0.057 |
+---------------+-------+
| IR_encoded(2) | 0.418 |
+---------------+-------+

CPD of EI_encoded
+---------------+---------------------+-----+---------------------+---------------------+
| IR_encoded    | IR_encoded(0)       | ... | IR_encoded(2)       | IR_encoded(2)       |
+---------------+---------------------+-----+---------------------+---------------------+
| SP_encoded    | SP_encoded(0)       | ... | SP_encoded(1)       | SP_encoded(2)       |
+---------------+---------------------+-----+---------------------+---------------------+
| EI_encoded(0) | 0.1986754966887417  | ... | 0.5                 | 0.3484848484848485  |
+---------------+---------------------+-----+-------------

...


Learned Structure (Edges) for 9000 samples (AIC):
[('IR_encoded', 'EI_encoded'), ('SP_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

AIC Score for 9000 samples: -25586.0556

CPD of IR_encoded
+---------------+----------------------+----------------------+---------------------+
| SP_encoded    | SP_encoded(0)        | SP_encoded(1)        | SP_encoded(2)       |
+---------------+----------------------+----------------------+---------------------+
| IR_encoded(0) | 0.307740130556419    | 0.5086916742909423   | 0.45726837060702874 |
+---------------+----------------------+----------------------+---------------------+
| IR_encoded(1) | 0.6673919801056886   | 0.46996035376639217  | 0.4972044728434505  |
+---------------+----------------------+----------------------+---------------------+
| IR_encoded(2) | 0.024867889337892447 | 0.021347971942665446 | 0.04552715654952077 |
+---------------+----------------------+----------------------+---------------------+

CPD of EI_encoded
+----

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 9500 samples (AIC):
[('IR_encoded', 'EI_encoded'), ('IR_encoded', 'SP_encoded'), ('SP_encoded', 'EI_encoded')]

AIC Score for 9500 samples: -28197.2232

CPD of IR_encoded
+---------------+----------+
| IR_encoded(0) | 0.196737 |
+---------------+----------+
| IR_encoded(1) | 0.319684 |
+---------------+----------+
| IR_encoded(2) | 0.483579 |
+---------------+----------+

CPD of EI_encoded
+---------------+----------------------+-----+---------------------+---------------------+
| IR_encoded    | IR_encoded(0)        | ... | IR_encoded(2)       | IR_encoded(2)       |
+---------------+----------------------+-----+---------------------+---------------------+
| SP_encoded    | SP_encoded(0)        | ... | SP_encoded(1)       | SP_encoded(2)       |
+---------------+----------------------+-----+---------------------+---------------------+
| EI_encoded(0) | 0.2945054945054945   | ... | 0.3809782608695652  | 0.42289239204934886 |
+---------------+-------------

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 10000 samples (AIC):
[('IR_encoded', 'EI_encoded'), ('IR_encoded', 'SP_encoded'), ('SP_encoded', 'EI_encoded')]

AIC Score for 10000 samples: -29385.4519

CPD of IR_encoded
+---------------+--------+
| IR_encoded(0) | 0.5033 |
+---------------+--------+
| IR_encoded(1) | 0.2318 |
+---------------+--------+
| IR_encoded(2) | 0.2649 |
+---------------+--------+

CPD of EI_encoded
+---------------+---------------------+-----+---------------------+---------------------+
| IR_encoded    | IR_encoded(0)       | ... | IR_encoded(2)       | IR_encoded(2)       |
+---------------+---------------------+-----+---------------------+---------------------+
| SP_encoded    | SP_encoded(0)       | ... | SP_encoded(1)       | SP_encoded(2)       |
+---------------+---------------------+-----+---------------------+---------------------+
| EI_encoded(0) | 0.03593339176161262 | ... | 0.12480252764612954 | 0.4421052631578947  |
+---------------+---------------------+-----+---

## BIC

In [ ]:
# Sample sizes to loop through
sample_sizes = range(500, 10500, 500)

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing sample size: {sample_size} (Sparse Data with BIC)")

    # Load the sparse dataset for the current sample size
    sparse_data_file = f'outcomes_sparse_{sample_size}.csv'
    df_sparse = pd.read_csv(sparse_data_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df_sparse['IR_encoded'] = df_sparse['IR'].map(ir_map)
    df_sparse['EI_encoded'] = df_sparse['EI'].map(ei_map)
    df_sparse['SP_encoded'] = df_sparse['SP'].map(sp_map)

    # Define the Hill-Climb structure learning algorithm
    hc_bic = HillClimbSearch(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded']])
    scoring_method_bic = BicScore(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded']])

    # Estimate the best structure using BIC
    best_dag_bic = hc_bic.estimate(scoring_method=scoring_method_bic)

    # Ensure all required nodes are present in the model, even if not connected
    best_model_bic = BayesianNetwork()
    best_model_bic.add_nodes_from(['IR_encoded', 'EI_encoded', 'SP_encoded'])  # Add all nodes
    best_model_bic.add_edges_from(best_dag_bic.edges())  # Add edges from the learned structure

    # Check if all nodes are included in the learned structure
    nodes_in_structure_bic = set(best_model_bic.nodes())
    required_nodes = {'IR_encoded', 'EI_encoded', 'SP_encoded'}

    if not required_nodes.issubset(nodes_in_structure_bic):
        print("\nNot all nodes are connected. Adding a dummy variable and ensuring all required nodes are present.")
        # Add a dummy variable to the dataset
        df_sparse['Dummy_Node'] = 1  # Constant dummy node

        # Ensure all required nodes are in the model by adding edges with the dummy node
        for node in required_nodes:
            if node not in nodes_in_structure_bic:
                best_model_bic.add_edge('Dummy_Node', node)

        # Re-estimate the structure with the dummy variable
        hc_bic = HillClimbSearch(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded', 'Dummy_Node']])
        scoring_method_bic = BicScore(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded', 'Dummy_Node']])
        best_dag_bic = hc_bic.estimate(scoring_method=scoring_method_bic)
        best_model_bic = BayesianNetwork(best_dag_bic.edges())

    # Display the learned structure (edges of the Bayesian Network)
    print(f"\nLearned Structure (Edges) for {sample_size} samples (BIC):")
    print(best_model_bic.edges())

    # Learn the CPDs using Maximum Likelihood Estimation (MLE)
    best_model_bic.fit(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded']], estimator=MaximumLikelihoodEstimator)

    # Check if the model is valid after learning the parameters
    assert best_model_bic.check_model()

    # Print the learned CPDs (Conditional Probability Distributions)
    for cpd in best_model_bic.get_cpds():
        print("\nCPD of", cpd.variable)
        print(cpd)

    # Create an inference object for the BIC model
    inference_bic = VariableElimination(best_model_bic)

    # Placeholder to store predictions
    predicted_sp_labels_bic = []

    # Loop through each row in the dataset to make predictions using BIC
    for index, row in df_sparse.iterrows():
        # Prepare the evidence from the dataset (IR_encoded and EI_encoded)
        sample_input = {'IR_encoded': int(row['IR_encoded']), 'EI_encoded': int(row['EI_encoded'])}

        # Perform inference to predict the distribution for SP_encoded (Stock Price)
        predicted_sp_distribution_bic = inference_bic.query(variables=['SP_encoded'], evidence=sample_input)

        # Extract the most likely SP_encoded class
        predicted_sp_class_bic = predicted_sp_distribution_bic.values.argmax()
        sp_reverse_map = {0: 'decrease', 1: 'stable', 2: 'increase'}
        predicted_sp_label_bic = sp_reverse_map[predicted_sp_class_bic]

        # Store the predicted label
        predicted_sp_labels_bic.append(predicted_sp_label_bic)

    # Convert the list of predicted labels into a DataFrame for easier comparison
    predicted_results_df_bic = pd.DataFrame({
        'IR': df_sparse['IR'],  # Original IR column
        'EI': df_sparse['EI'],  # Original EI column
        'Predicted_SP': predicted_sp_labels_bic  # Predicted SP column
    })

    # Add the actual SP values for comparison
    predicted_results_df_bic['Actual_SP'] = df_sparse['SP']

    # Calculate accuracy of predictions for BIC
    accuracy_bic = accuracy_score(predicted_results_df_bic['Actual_SP'], predicted_results_df_bic['Predicted_SP'])
    print(f"\nPrediction Accuracy for {sample_size} samples (BIC): {accuracy_bic:.4f}")

    # Display the first few rows of predictions for BIC
    print(f"\nPredicted Results for Sparse Data (First 10 rows) for {sample_size} samples (BIC):")
    print(predicted_results_df_bic.head(10))

    # Calculate the BIC score for the Bayesian Network model
    bic_score_value = scoring_method_bic.score(best_model_bic)

    # Print the BIC score
    print(f"\nBIC Score for {sample_size} samples: {bic_score_value:.4f}")

# Notify the user that the process is done
print("\nProcessing complete for all sample sizes using BIC (Sparse Data).")


Processing sample size: 500 (Sparse Data with BIC)


  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 500 samples (BIC):
[('SP_encoded', 'EI_encoded')]

CPD of IR_encoded
+---------------+------+
| IR_encoded(0) | 0.44 |
+---------------+------+
| IR_encoded(1) | 0.49 |
+---------------+------+
| IR_encoded(2) | 0.07 |
+---------------+------+

CPD of EI_encoded
+---------------+---------------------+--------------------+---------------------+
| SP_encoded    | SP_encoded(0)       | SP_encoded(1)      | SP_encoded(2)       |
+---------------+---------------------+--------------------+---------------------+
| EI_encoded(0) | 0.3248407643312102  | 0.2222222222222222 | 0.3768844221105528  |
+---------------+---------------------+--------------------+---------------------+
| EI_encoded(1) | 0.27388535031847133 | 0.2013888888888889 | 0.4623115577889447  |
+---------------+---------------------+--------------------+---------------------+
| EI_encoded(2) | 0.4012738853503185  | 0.5763888888888888 | 0.16080402010050251 |
+---------------+---------------------+---

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 1000 samples (BIC):
[('EI_encoded', 'SP_encoded'), ('SP_encoded', 'IR_encoded')]

CPD of IR_encoded
+---------------+----------------------+---------------------+---------------------+
| SP_encoded    | SP_encoded(0)        | SP_encoded(1)       | SP_encoded(2)       |
+---------------+----------------------+---------------------+---------------------+
| IR_encoded(0) | 0.037037037037037035 | 0.14285714285714285 | 0.13649851632047477 |
+---------------+----------------------+---------------------+---------------------+
| IR_encoded(1) | 0.2152777777777778   | 0.46320346320346323 | 0.2818991097922849  |
+---------------+----------------------+---------------------+---------------------+
| IR_encoded(2) | 0.7476851851851852   | 0.3939393939393939  | 0.5816023738872403  |
+---------------+----------------------+---------------------+---------------------+

CPD of EI_encoded
+---------------+-------+
| EI_encoded(0) | 0.395 |
+---------------+-------+
| EI_en

...


Learned Structure (Edges) for 9000 samples (BIC):
[('IR_encoded', 'EI_encoded'), ('IR_encoded', 'SP_encoded'), ('SP_encoded', 'EI_encoded')]

CPD of IR_encoded
+---------------+----------+
| IR_encoded(0) | 0.177889 |
+---------------+----------+
| IR_encoded(1) | 0.125    |
+---------------+----------+
| IR_encoded(2) | 0.697111 |
+---------------+----------+

CPD of EI_encoded
+---------------+--------------------+-----+---------------------+--------------------+
| IR_encoded    | IR_encoded(0)      | ... | IR_encoded(2)       | IR_encoded(2)      |
+---------------+--------------------+-----+---------------------+--------------------+
| SP_encoded    | SP_encoded(0)      | ... | SP_encoded(1)       | SP_encoded(2)      |
+---------------+--------------------+-----+---------------------+--------------------+
| EI_encoded(0) | 0.0860655737704918 | ... | 0.4263653483992467  | 0.466710182767624  |
+---------------+--------------------+-----+---------------------+--------------------+
|

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 9500 samples (BIC):
[('IR_encoded', 'SP_encoded'), ('EI_encoded', 'SP_encoded')]

CPD of IR_encoded
+---------------+----------+
| IR_encoded(0) | 0.362947 |
+---------------+----------+
| IR_encoded(1) | 0.288632 |
+---------------+----------+
| IR_encoded(2) | 0.348421 |
+---------------+----------+

CPD of EI_encoded
+---------------+----------+
| EI_encoded(0) | 0.288421 |
+---------------+----------+
| EI_encoded(1) | 0.313684 |
+---------------+----------+
| EI_encoded(2) | 0.397895 |
+---------------+----------+

CPD of SP_encoded
+---------------+----------------------+-----+---------------------+--------------------+
| EI_encoded    | EI_encoded(0)        | ... | EI_encoded(2)       | EI_encoded(2)      |
+---------------+----------------------+-----+---------------------+--------------------+
| IR_encoded    | IR_encoded(0)        | ... | IR_encoded(1)       | IR_encoded(2)      |
+---------------+----------------------+-----+-------------------

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 10000 samples (BIC):
[('EI_encoded', 'IR_encoded'), ('SP_encoded', 'IR_encoded'), ('SP_encoded', 'EI_encoded')]

CPD of IR_encoded
+---------------+---------------------+-----+---------------------+---------------------+
| EI_encoded    | EI_encoded(0)       | ... | EI_encoded(2)       | EI_encoded(2)       |
+---------------+---------------------+-----+---------------------+---------------------+
| SP_encoded    | SP_encoded(0)       | ... | SP_encoded(1)       | SP_encoded(2)       |
+---------------+---------------------+-----+---------------------+---------------------+
| IR_encoded(0) | 0.22929936305732485 | ... | 0.37530864197530867 | 0.41580756013745707 |
+---------------+---------------------+-----+---------------------+---------------------+
| IR_encoded(1) | 0.554140127388535   | ... | 0.4765432098765432  | 0.5189003436426117  |
+---------------+---------------------+-----+---------------------+---------------------+
| IR_encoded(2) | 0.21656050

## AIC

In [ ]:
# Sample sizes to loop through
sample_sizes = range(500, 10500, 500)

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing sample size: {sample_size} (Sparse Data with AIC)")

    # Load the sparse dataset for the current sample size
    sparse_data_file = f'outcomes_sparse_{sample_size}.csv'
    df_sparse = pd.read_csv(sparse_data_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df_sparse['IR_encoded'] = df_sparse['IR'].map(ir_map)
    df_sparse['EI_encoded'] = df_sparse['EI'].map(ei_map)
    df_sparse['SP_encoded'] = df_sparse['SP'].map(sp_map)

    # Define the Hill-Climb structure learning algorithm
    hc_aic = HillClimbSearch(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded']])
    scoring_method_aic = AICScore(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded']])

    # Estimate the best structure using AIC
    best_dag_aic = hc_aic.estimate(scoring_method=scoring_method_aic)

    # Ensure all required nodes are present in the model, even if not connected
    best_model_aic = BayesianNetwork()
    best_model_aic.add_nodes_from(['IR_encoded', 'EI_encoded', 'SP_encoded'])  # Add all nodes
    best_model_aic.add_edges_from(best_dag_aic.edges())  # Add edges from the learned structure

    # Check if all nodes are included in the learned structure
    nodes_in_structure_aic = set(best_model_aic.nodes())
    required_nodes = {'IR_encoded', 'EI_encoded', 'SP_encoded'}

    if not required_nodes.issubset(nodes_in_structure_aic):
        print("\nNot all nodes are connected. Adding a dummy variable and ensuring all required nodes are present.")
        # Add a dummy variable to the dataset
        df_sparse['Dummy_Node'] = 1  # Constant dummy node

        # Ensure all required nodes are in the model by adding edges with the dummy node
        for node in required_nodes:
            if node not in nodes_in_structure_aic:
                best_model_aic.add_edge('Dummy_Node', node)

        # Re-estimate the structure with the dummy variable
        hc_aic = HillClimbSearch(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded', 'Dummy_Node']])
        scoring_method_aic = AICScore(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded', 'Dummy_Node']])
        best_dag_aic = hc_aic.estimate(scoring_method=scoring_method_aic)
        best_model_aic = BayesianNetwork(best_dag_aic.edges())

    # Display the learned structure (edges of the Bayesian Network)
    print(f"\nLearned Structure (Edges) for {sample_size} samples (AIC):")
    print(best_model_aic.edges())

    # Learn the CPDs using Maximum Likelihood Estimation (MLE)
    best_model_aic.fit(df_sparse[['IR_encoded', 'EI_encoded', 'SP_encoded']], estimator=MaximumLikelihoodEstimator)

    # Check if the model is valid after learning the parameters
    assert best_model_aic.check_model()

    # Print the learned CPDs (Conditional Probability Distributions)
    for cpd in best_model_aic.get_cpds():
        print("\nCPD of", cpd.variable)
        print(cpd)

    # Create an inference object for the AIC model
    inference_aic = VariableElimination(best_model_aic)

    # Placeholder to store predictions
    predicted_sp_labels_aic = []

    # Loop through each row in the dataset to make predictions using AIC
    for index, row in df_sparse.iterrows():
        # Prepare the evidence from the dataset (IR_encoded and EI_encoded)
        sample_input = {'IR_encoded': int(row['IR_encoded']), 'EI_encoded': int(row['EI_encoded'])}

        # Perform inference to predict the distribution for SP_encoded (Stock Price)
        predicted_sp_distribution_aic = inference_aic.query(variables=['SP_encoded'], evidence=sample_input)

        # Extract the most likely SP_encoded class
        predicted_sp_class_aic = predicted_sp_distribution_aic.values.argmax()
        sp_reverse_map = {0: 'decrease', 1: 'stable', 2: 'increase'}
        predicted_sp_label_aic = sp_reverse_map[predicted_sp_class_aic]

        # Store the predicted label
        predicted_sp_labels_aic.append(predicted_sp_label_aic)

    # Convert the list of predicted labels into a DataFrame for easier comparison
    predicted_results_df_aic = pd.DataFrame({
        'IR': df_sparse['IR'],  # Original IR column
        'EI': df_sparse['EI'],  # Original EI column
        'Predicted_SP': predicted_sp_labels_aic  # Predicted SP column
    })

    # Add the actual SP values for comparison
    predicted_results_df_aic['Actual_SP'] = df_sparse['SP']

    # Calculate accuracy of predictions for AIC
    accuracy_aic = accuracy_score(predicted_results_df_aic['Actual_SP'], predicted_results_df_aic['Predicted_SP'])
    print(f"\nPrediction Accuracy for {sample_size} samples (AIC): {accuracy_aic:.4f}")

    # Display the first few rows of predictions for AIC
    print(f"\nPredicted Results for Sparse Data (First 10 rows) for {sample_size} samples (AIC):")
    print(predicted_results_df_aic.head(10))

    # Calculate the AIC score for the Bayesian Network model
    aic_score_value = scoring_method_aic.score(best_model_aic)

    # Print the AIC score
    print(f"\nAIC Score for {sample_size} samples: {aic_score_value:.4f}")

# Notify the user that the process is done
print("\nProcessing complete for all sample sizes using AIC (Sparse Data).")


Processing sample size: 500 (Sparse Data with AIC)


  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 500 samples (AIC):
[('IR_encoded', 'EI_encoded'), ('SP_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

CPD of IR_encoded
+---------------+---------------------+---------------------+---------------------+
| SP_encoded    | SP_encoded(0)       | SP_encoded(1)       | SP_encoded(2)       |
+---------------+---------------------+---------------------+---------------------+
| IR_encoded(0) | 0.34394904458598724 | 0.375               | 0.5628140703517588  |
+---------------+---------------------+---------------------+---------------------+
| IR_encoded(1) | 0.5923566878980892  | 0.5347222222222222  | 0.3768844221105528  |
+---------------+---------------------+---------------------+---------------------+
| IR_encoded(2) | 0.06369426751592357 | 0.09027777777777778 | 0.06030150753768844 |
+---------------+---------------------+---------------------+---------------------+

CPD of EI_encoded
+---------------+---------------------+-----+--------------------

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 1000 samples (AIC):
[('IR_encoded', 'SP_encoded'), ('EI_encoded', 'SP_encoded')]

CPD of IR_encoded
+---------------+-------+
| IR_encoded(0) | 0.095 |
+---------------+-------+
| IR_encoded(1) | 0.295 |
+---------------+-------+
| IR_encoded(2) | 0.61  |
+---------------+-------+

CPD of EI_encoded
+---------------+-------+
| EI_encoded(0) | 0.395 |
+---------------+-------+
| EI_encoded(1) | 0.311 |
+---------------+-------+
| EI_encoded(2) | 0.294 |
+---------------+-------+

CPD of SP_encoded
+---------------+---------------------+-----+---------------------+---------------------+
| EI_encoded    | EI_encoded(0)       | ... | EI_encoded(2)       | EI_encoded(2)       |
+---------------+---------------------+-----+---------------------+---------------------+
| IR_encoded    | IR_encoded(0)       | ... | IR_encoded(1)       | IR_encoded(2)       |
+---------------+---------------------+-----+---------------------+---------------------+
| SP_encoded(0) |

...


Learned Structure (Edges) for 9000 samples (AIC):
[('IR_encoded', 'EI_encoded'), ('IR_encoded', 'SP_encoded'), ('SP_encoded', 'EI_encoded')]

CPD of IR_encoded
+---------------+----------+
| IR_encoded(0) | 0.177889 |
+---------------+----------+
| IR_encoded(1) | 0.125    |
+---------------+----------+
| IR_encoded(2) | 0.697111 |
+---------------+----------+

CPD of EI_encoded
+---------------+--------------------+-----+---------------------+--------------------+
| IR_encoded    | IR_encoded(0)      | ... | IR_encoded(2)       | IR_encoded(2)      |
+---------------+--------------------+-----+---------------------+--------------------+
| SP_encoded    | SP_encoded(0)      | ... | SP_encoded(1)       | SP_encoded(2)      |
+---------------+--------------------+-----+---------------------+--------------------+
| EI_encoded(0) | 0.0860655737704918 | ... | 0.4263653483992467  | 0.466710182767624  |
+---------------+--------------------+-----+---------------------+--------------------+
|

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 9500 samples (AIC):
[('IR_encoded', 'SP_encoded'), ('IR_encoded', 'EI_encoded'), ('EI_encoded', 'SP_encoded')]

CPD of IR_encoded
+---------------+----------+
| IR_encoded(0) | 0.362947 |
+---------------+----------+
| IR_encoded(1) | 0.288632 |
+---------------+----------+
| IR_encoded(2) | 0.348421 |
+---------------+----------+

CPD of EI_encoded
+---------------+---------------------+---------------------+---------------------+
| IR_encoded    | IR_encoded(0)       | IR_encoded(1)       | IR_encoded(2)       |
+---------------+---------------------+---------------------+---------------------+
| EI_encoded(0) | 0.2827726218097448  | 0.29285193289569655 | 0.29063444108761327 |
+---------------+---------------------+---------------------+---------------------+
| EI_encoded(1) | 0.31322505800464034 | 0.2975929978118162  | 0.3274924471299094  |
+---------------+---------------------+---------------------+---------------------+
| EI_encoded(2) | 0.404002320

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 10000 samples (AIC):
[('EI_encoded', 'IR_encoded'), ('SP_encoded', 'IR_encoded'), ('SP_encoded', 'EI_encoded')]

CPD of IR_encoded
+---------------+---------------------+-----+---------------------+---------------------+
| EI_encoded    | EI_encoded(0)       | ... | EI_encoded(2)       | EI_encoded(2)       |
+---------------+---------------------+-----+---------------------+---------------------+
| SP_encoded    | SP_encoded(0)       | ... | SP_encoded(1)       | SP_encoded(2)       |
+---------------+---------------------+-----+---------------------+---------------------+
| IR_encoded(0) | 0.22929936305732485 | ... | 0.37530864197530867 | 0.41580756013745707 |
+---------------+---------------------+-----+---------------------+---------------------+
| IR_encoded(1) | 0.554140127388535   | ... | 0.4765432098765432  | 0.5189003436426117  |
+---------------+---------------------+-----+---------------------+---------------------+
| IR_encoded(2) | 0.21656050

# ------------------------------------------------------------------------------------------------------------

# Hypothesis Model 500, 1000, 1500, ..., 10000 Samples (dense) 1 hidden Layer, 10 Neurons

In [5]:
# Sample sizes to loop through
sample_sizes = range(500, 10500, 500)

# Define the Neural Network architecture
def create_nn_model(hidden_layers=1, nodes_per_layer=10):
    model = models.Sequential()

    # Input layer (2 input features: IR_encoded and EI_encoded)
    model.add(layers.InputLayer(input_shape=(2,)))

    # Hidden layers
    for layer_num in range(hidden_layers):
        model.add(layers.Dense(nodes_per_layer, activation='relu', name=f"hidden_layer_{layer_num + 1}"))

    # Output layer (3 classes: decrease, stable, increase)
    model.add(layers.Dense(3, activation='softmax', name="output_layer"))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Loop through each sample size
for size in sample_sizes:
    # Load data for the current sample size (adjust the file paths if necessary)
    outcomes_file = f'outcomes_dense_{size}.csv'

    df = pd.read_csv(outcomes_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df['IR_encoded'] = df['IR'].map(ir_map)
    df['EI_encoded'] = df['EI'].map(ei_map)
    df['SP_encoded'] = df['SP'].map(sp_map)

    # Features (IR and EI) and labels (SP)
    X = df[['IR_encoded', 'EI_encoded']]
    y = df['SP_encoded']

    # Refresh the data split for each iteration
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False, random_state=42)

    # Show split confirmation
    print(f"\nSample size: {size}")
    print("Training Data:", X_train.shape, y_train.shape)
    print("Validation Data:", X_val.shape, y_val.shape)
    print("Test Data:", X_test.shape, y_test.shape)

    # Create the Neural Network model
    nn_model = create_nn_model(hidden_layers=1, nodes_per_layer=10)

    # Train the model
    history = nn_model.fit(X_train, y_train,
                           epochs=50,
                           batch_size=32,
                           validation_data=(X_val, y_val),
                           verbose=0)  # Set verbose=0 to avoid too much output

    # Evaluate on the validation set
    val_loss, val_accuracy = nn_model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Accuracy for {size} samples: {val_accuracy:.4f}")

    # Evaluate on the test set
    test_loss, test_accuracy = nn_model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy for {size} samples: {test_accuracy:.4f}")

    # Make predictions on the test set
    predictions = nn_model.predict(X_test)

    # Convert the predicted probabilities to class labels
    predicted_classes = predictions.argmax(axis=1)

    # Create a list to map integers back to the original SP labels
    sp_reverse_map = ['decrease', 'stable', 'increase']

    # Convert the predicted classes to the original labels
    predicted_labels = [sp_reverse_map[label] for label in predicted_classes]

    # Create a DataFrame for the predicted probabilities
    probs_df = pd.DataFrame(predictions, columns=['Prob_decrease', 'Prob_stable', 'Prob_increase'])

    # Output the IR, EI, predicted SP, and the NN probabilities
    result_df = pd.DataFrame({
        'IR': df['IR'][:len(predicted_labels)],  # IR column from the original dataframe
        'EI': df['EI'][:len(predicted_labels)],  # EI column from the original dataframe
        'Predicted_SP': predicted_labels         # Predicted SP labels
    })

    # Combine the result with the predicted probabilities
    combined_df = pd.concat([result_df, probs_df.reset_index(drop=True)], axis=1)

    # Show the first few rows of the results for this sample size
    print(f"\nPredicted Results and Probabilities for {size} samples (First 5 rows):")
    print(combined_df.head(15))

# After the loop is done, print this message
print("\nLooping through all sample sizes complete!")


Sample size: 500
Training Data: (350, 2) (350,)
Validation Data: (75, 2) (75,)
Test Data: (75, 2) (75,)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 500 samples: 0.4400
Test Accuracy for 500 samples: 0.3600
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

Predicted Results and Probabilities for 500 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0      low     good       stable       0.220026     0.416640       0.363334
1      low  average     decrease       0.413598     0.217326       0.369076
2     high     good       stable       0.220026     0.416640       0.363334
3   medium  average     increase       0.284828     0.339500       0.375672
4      low     good     decrease       0.482076     0.230441       0.287483
5      low  average     decrease       0.413598     0.217326       0.369076
6      low  average     decrease       0.402544     0.247264       0.350192
7     high     poor       stable       0.220026     0.416640       0.363334
8      low     good     decrease       0.382125     0.254997       0.362877
9      low     good     decrease       0.397598    

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 1000 samples: 0.3800
Test Accuracy for 1000 samples: 0.3867
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 

Predicted Results and Probabilities for 1000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0   medium     poor     decrease       0.484780     0.186481       0.328738
1      low  average     increase       0.319682     0.272961       0.407357
2   medium     poor     decrease       0.484780     0.186481       0.328738
3     high     poor     increase       0.319682     0.272961       0.407357
4     high  average     decrease       0.458092     0.244180       0.297728
5   medium     good     increase       0.199729     0.298456       0.501814
6      low     poor     increase       0.270419     0.318913       0.410667
7     high     poor     decrease       0.458092     0.244180       0.297728
8   medium     good     increase       0.341192     0.244760       0.414048
9     high     good     decrease       0.484780 

...

Validation Accuracy for 9000 samples: 0.5785
Test Accuracy for 9000 samples: 0.5422
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 9000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0      low     poor     increase       0.203553     0.130800       0.665647
1   medium  average     increase       0.203553     0.130800       0.665647
2     high     poor     increase       0.203553     0.130800       0.665647
3   medium     poor     increase       0.203553     0.130800       0.665647
4   medium     poor     increase       0.332343     0.109771       0.557886
5   medium     good     increase       0.332343     0.109771       0.557886
6   medium     good     increase       0.203553     0.130800       0.665647
7   medium  average     increase       0.203553     0.130800       0.665647
8   medium     poor     increase       0.323144     0.178940       0.497915
9   medium     poor     increase       0.332343

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 9500 samples: 0.5846
Test Accuracy for 9500 samples: 0.5432
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 9500 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0      low  average       stable       0.205373     0.542198       0.252428
1      low     good       stable       0.205373     0.542198       0.252428
2     high  average     increase       0.039831     0.201106       0.759062
3     high  average     increase       0.039831     0.201106       0.759062
4   medium  average     increase       0.039831     0.201106       0.759062
5     high     poor       stable       0.361559     0.440328       0.198113
6     high     poor       stable       0.361559     0.440328       0.198113
7      low     poor       stable       0.361559     0.440328       0.198113
8      low     poor       stable       0.361559     0.440328       0.198113
9      low     good       stable       0.361559

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 10000 samples: 0.5120
Test Accuracy for 10000 samples: 0.5407
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 10000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0      low  average     increase       0.359770     0.169999       0.470231
1   medium  average     decrease       0.526740     0.162752       0.310508
2   medium     good     increase       0.089461     0.451430       0.459109
3   medium     good     decrease       0.526740     0.162752       0.310508
4     high     poor     increase       0.309916     0.051491       0.638593
5     high     poor     increase       0.122031     0.430991       0.446978
6     high     good     increase       0.122031     0.430991       0.446978
7     high  average       stable       0.236517     0.452659       0.310824
8     high  average     increase       0.122031     0.430991       0.446978
9   medium     good     increase       0.122

# Hypothesis Model 500, 1000, 1500, ..., 10000 Samples (sparse) 1 hidden Layer, 10 Neurons

In [6]:
# Sample sizes to loop through
sample_sizes = range(500, 10500, 500)

# Define the Neural Network architecture
def create_nn_model(hidden_layers=1, nodes_per_layer=10):
    model = models.Sequential()

    # Input layer (2 input features: IR_encoded and EI_encoded)
    model.add(layers.InputLayer(input_shape=(2,)))

    # Hidden layers
    for layer_num in range(hidden_layers):
        model.add(layers.Dense(nodes_per_layer, activation='relu', name=f"hidden_layer_{layer_num + 1}"))

    # Output layer (3 classes: decrease, stable, increase)
    model.add(layers.Dense(3, activation='softmax', name="output_layer"))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Loop through each sample size
for size in sample_sizes:
    # Load data for the current sample size (adjust the file paths for sparse data)
    outcomes_file = f'outcomes_sparse_{size}.csv'

    df = pd.read_csv(outcomes_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df['IR_encoded'] = df['IR'].map(ir_map)
    df['EI_encoded'] = df['EI'].map(ei_map)
    df['SP_encoded'] = df['SP'].map(sp_map)

    # Features (IR and EI) and labels (SP)
    X = df[['IR_encoded', 'EI_encoded']]
    y = df['SP_encoded']

    # Refresh the data split for each iteration
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False, random_state=42)

    # Show split confirmation
    print(f"\nSample size: {size}")
    print("Training Data:", X_train.shape, y_train.shape)
    print("Validation Data:", X_val.shape, y_val.shape)
    print("Test Data:", X_test.shape, y_test.shape)

    # Create the Neural Network model
    nn_model = create_nn_model(hidden_layers=1, nodes_per_layer=10)

    # Train the model
    history = nn_model.fit(X_train, y_train,
                           epochs=50,
                           batch_size=32,
                           validation_data=(X_val, y_val),
                           verbose=0)  # Set verbose=0 to avoid too much output

    # Evaluate on the validation set
    val_loss, val_accuracy = nn_model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Accuracy for {size} samples: {val_accuracy:.4f}")

    # Evaluate on the test set
    test_loss, test_accuracy = nn_model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy for {size} samples: {test_accuracy:.4f}")

    # Make predictions on the test set
    predictions = nn_model.predict(X_test)

    # Convert the predicted probabilities to class labels
    predicted_classes = predictions.argmax(axis=1)

    # Create a list to map integers back to the original SP labels
    sp_reverse_map = ['decrease', 'stable', 'increase']

    # Convert the predicted classes to the original labels
    predicted_labels = [sp_reverse_map[label] for label in predicted_classes]

    # Create a DataFrame for the predicted probabilities
    probs_df = pd.DataFrame(predictions, columns=['Prob_decrease', 'Prob_stable', 'Prob_increase'])

    # Output the IR, EI, predicted SP, and the NN probabilities
    result_df = pd.DataFrame({
        'IR': df['IR'][:len(predicted_labels)],  # IR column from the original dataframe
        'EI': df['EI'][:len(predicted_labels)],  # EI column from the original dataframe
        'Predicted_SP': predicted_labels         # Predicted SP labels
    })

    # Combine the result with the predicted probabilities
    combined_df = pd.concat([result_df, probs_df.reset_index(drop=True)], axis=1)

    # Show the first few rows of the results for this sample size
    print(f"\nPredicted Results and Probabilities for {size} samples (First 5 rows):")
    print(combined_df.head(15))

# After the loop is done, print this message
print("\nLooping through all sparse sample sizes complete!")


Sample size: 500
Training Data: (350, 2) (350,)
Validation Data: (75, 2) (75,)
Test Data: (75, 2) (75,)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 500 samples: 0.4533
Test Accuracy for 500 samples: 0.4000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

Predicted Results and Probabilities for 500 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0     high     poor     increase       0.354493     0.236941       0.408565
1     high     good     increase       0.354493     0.236941       0.408565
2   medium  average       stable       0.387208     0.391846       0.220946
3   medium     good       stable       0.393601     0.469468       0.136931
4     high  average     decrease       0.400182     0.297971       0.301847
5     high  average       stable       0.393601     0.469468       0.136931
6      low     good       stable       0.393601     0.469468       0.136931
7     high  average     increase       0.307705     0.336684       0.355610
8      low  average     decrease       0.400182     0.297971       0.301847
9   medium  average     decrease       0.400182    

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 1000 samples: 0.4600
Test Accuracy for 1000 samples: 0.5400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

Predicted Results and Probabilities for 1000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0   medium     good     increase       0.319657     0.316506       0.363837
1   medium     good     increase       0.397934     0.169565       0.432501
2     high  average     decrease       0.452440     0.250053       0.297507
3     high  average     decrease       0.382008     0.329299       0.288693
4      low     good     decrease       0.388275     0.361029       0.250696
5     high     good     increase       0.397934     0.169565       0.432501
6   medium     good     decrease       0.388275     0.361029       0.250696
7   medium  average     increase       0.397934     0.169565       0.432501
8     high     good     decrease       0.388275     0.361029       0.250696
9     high     poor     decrease       0.388275 

...

Validation Accuracy for 9000 samples: 0.4422
Test Accuracy for 9000 samples: 0.4519
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 9000 samples (First 5 rows):
      IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0   high     good     decrease       0.390754     0.376969       0.232277
1   high  average     decrease       0.390754     0.376969       0.232277
2   high     poor       stable       0.421453     0.503074       0.075473
3   high  average     decrease       0.390754     0.376969       0.232277
4   high  average     decrease       0.390754     0.376969       0.232277
5   high     good       stable       0.421453     0.503074       0.075473
6   high     good       stable       0.421453     0.503074       0.075473
7   high     good     decrease       0.383619     0.278086       0.338295
8   high     good     decrease       0.390754     0.376969       0.232277
9   high     poor     decrease       0.383619     0.278086       0.

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 9500 samples: 0.5018
Test Accuracy for 9500 samples: 0.4912
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 9500 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0     high     poor       stable       0.094702     0.587856       0.317441
1     high     good       stable       0.322751     0.381096       0.296153
2   medium     poor     increase       0.298169     0.251321       0.450510
3   medium     poor     increase       0.216407     0.314835       0.468758
4   medium     poor     increase       0.133787     0.201980       0.664233
5      low     poor     increase       0.216407     0.314835       0.468758
6     high     poor     decrease       0.398109     0.331615       0.270276
7   medium     poor     increase       0.216407     0.314835       0.468758
8   medium     poor     increase       0.133787     0.201980       0.664233
9     high     poor     increase       0.216407

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 10000 samples: 0.4827
Test Accuracy for 10000 samples: 0.4720
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 10000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0      low     good     decrease       0.495838     0.097371       0.406792
1      low  average       stable       0.314306     0.462680       0.223013
2     high     good       stable       0.314306     0.462680       0.223013
3      low     good     decrease       0.495838     0.097371       0.406792
4     high     good     decrease       0.495838     0.097371       0.406792
5     high  average     decrease       0.404043     0.309526       0.286431
6     high  average       stable       0.211935     0.691541       0.096524
7      low  average     decrease       0.495838     0.097371       0.406792
8      low     good       stable       0.314306     0.462680       0.223013
9   medium     good     decrease       0.404

# ------------------------------------------------------------------------------------------------------------

# K-L Divergence NN Dense Data

## This is the entropy for dense data distribution with a sample size of 500 - 10000 and a NN with 1 hidden layer and 10 Neurons.

In [12]:
# Define the function to save K-L divergence to a file at the end
def save_kl_divergences_to_file(kl_divergence_data):
    file_name = 'K-L Divergence dense NN 1_10.csv'

    # Save the K-L divergences to a CSV file
    with open(file_name, 'w') as f:
        f.write('Size,NN_Dense_1_10_Entropy\n')  # Write the headers
        for sample_size, kl_div_value in kl_divergence_data.items():
            f.write(f"{sample_size},{kl_div_value:.4f}\n")

# Placeholder to store K-L divergence values for all sample sizes
kl_divergence_results = {}

# Sample sizes to loop through
sample_sizes = range(500, 10500, 500)

# Loop through each sample size
for sample_size in sample_sizes:
    # Load the ground truth probabilities for the current sample size
    ground_truth_probs_file = f'probabilities_dense_{sample_size}.csv'
    df_gt_probs = pd.read_csv(ground_truth_probs_file)

    # Load outcomes for the current sample size
    outcomes_file = f'outcomes_dense_{sample_size}.csv'
    df = pd.read_csv(outcomes_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}

    df['IR_encoded'] = df['IR'].map(ir_map)
    df['EI_encoded'] = df['EI'].map(ei_map)

    # Placeholder to store K-L divergence values
    kl_divergences = []

    # Loop through the test set predictions
    for i in range(len(df)):
        # Neural Network predicted probabilities for SP (decrease, stable, increase)
        predicted_probs = predictions[i]  # Assuming the NN predictions are already available in memory for this sample size

        # Get the IR and EI values for the current sample
        ir_value = df.iloc[i]['IR_encoded']
        ei_value = df.iloc[i]['EI_encoded']

        # Map encoded values back to original labels
        ir_value = {0: 'low', 1: 'medium', 2: 'high'}[ir_value]
        ei_value = {0: 'poor', 1: 'average', 2: 'good'}[ei_value]

        # Get the corresponding ground truth probabilities for SP given IR and EI
        col_prefix = f'SP_given_IR_{ir_value}_EI_{ei_value}'
        ground_truth_probs = df_gt_probs.filter(like=col_prefix).values.flatten()

        # Ensure the probabilities are non-zero to avoid division by zero
        epsilon = 1e-10
        ground_truth_probs = np.clip(ground_truth_probs, epsilon, 1)

        # Compute K-L divergence (Neural Network vs Ground Truth)
        kl_div = entropy(predicted_probs, ground_truth_probs)
        kl_divergences.append(kl_div)

    # Calculate the average K-L divergence over all samples for the current sample size
    average_kl_divergence = np.mean(kl_divergences)

    # Store the result for this sample size
    kl_divergence_results[sample_size] = average_kl_divergence

    # Print confirmation for each sample size
    print(f"Average K-L Divergence for {sample_size} dense samples: {average_kl_divergence:.4f}")

# Once all sample sizes are processed, save the results to a CSV file
save_kl_divergences_to_file(kl_divergence_results)

# Print completion message
print("\nK-L divergence calculations complete and saved to 'K-L Divergence dense NN 1_10.csv'.")

Average K-L Divergence for 500 dense samples: 0.3219
Average K-L Divergence for 1000 dense samples: 0.4108
Average K-L Divergence for 1500 dense samples: 0.4250
Average K-L Divergence for 2000 dense samples: 0.3801
Average K-L Divergence for 2500 dense samples: 0.3694
Average K-L Divergence for 3000 dense samples: 0.4624
Average K-L Divergence for 3500 dense samples: 0.4288
Average K-L Divergence for 4000 dense samples: 0.4070
Average K-L Divergence for 4500 dense samples: 0.2583
Average K-L Divergence for 5000 dense samples: 0.3882
Average K-L Divergence for 5500 dense samples: 0.2103
Average K-L Divergence for 6000 dense samples: 0.6866
Average K-L Divergence for 6500 dense samples: 0.2862
Average K-L Divergence for 7000 dense samples: 0.2105
Average K-L Divergence for 7500 dense samples: 0.2742
Average K-L Divergence for 8000 dense samples: 0.6441
Average K-L Divergence for 8500 dense samples: 0.2623
Average K-L Divergence for 9000 dense samples: 0.3623
Average K-L Divergence for 95

# K-L Divergence NN Sparse Data

## This is the entropy for sparse data distribution with a sample size of 500 - 10000 and a NN with 1 hidden layer and 10 Neurons.

In [13]:
# Define the function to save K-L divergence to a file at the end
def save_kl_divergences_to_file(kl_divergence_data):
    file_name = 'K-L Divergence sparse NN 1_10.csv'  # Changed file name to indicate sparse data

    # Save the K-L divergences to a CSV file
    with open(file_name, 'w') as f:
        f.write('Size,NN_Sparse_1_10_Entropy\n')  # Updated the header to indicate sparse data
        for sample_size, kl_div_value in kl_divergence_data.items():
            f.write(f"{sample_size},{kl_div_value:.4f}\n")

# Placeholder to store K-L divergence values for all sample sizes
kl_divergence_results = {}

# Sample sizes to loop through
sample_sizes = range(500, 10500, 500)

# Loop through each sample size
for sample_size in sample_sizes:
    # Load the ground truth probabilities for the current sample size (sparse data)
    ground_truth_probs_file = f'probabilities_sparse_{sample_size}.csv'  # Changed to sparse data file
    df_gt_probs = pd.read_csv(ground_truth_probs_file)

    # Load outcomes for the current sample size (sparse data)
    outcomes_file = f'outcomes_sparse_{sample_size}.csv'  # Changed to sparse data file
    df = pd.read_csv(outcomes_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}

    df['IR_encoded'] = df['IR'].map(ir_map)
    df['EI_encoded'] = df['EI'].map(ei_map)

    # Placeholder to store K-L divergence values
    kl_divergences = []

    # Loop through the test set predictions
    for i in range(len(df)):
        # Neural Network predicted probabilities for SP (decrease, stable, increase)
        predicted_probs = predictions[i]  # Assuming the NN predictions are already available in memory for this sample size

        # Get the IR and EI values for the current sample
        ir_value = df.iloc[i]['IR_encoded']
        ei_value = df.iloc[i]['EI_encoded']

        # Map encoded values back to original labels
        ir_value = {0: 'low', 1: 'medium', 2: 'high'}[ir_value]
        ei_value = {0: 'poor', 1: 'average', 2: 'good'}[ei_value]

        # Get the corresponding ground truth probabilities for SP given IR and EI
        col_prefix = f'SP_given_IR_{ir_value}_EI_{ei_value}'
        ground_truth_probs = df_gt_probs.filter(like=col_prefix).values.flatten()

        # Ensure the probabilities are non-zero to avoid division by zero
        epsilon = 1e-10
        ground_truth_probs = np.clip(ground_truth_probs, epsilon, 1)

        # Compute K-L divergence (Neural Network vs Ground Truth)
        kl_div = entropy(predicted_probs, ground_truth_probs)
        kl_divergences.append(kl_div)

    # Calculate the average K-L divergence over all samples for the current sample size
    average_kl_divergence = np.mean(kl_divergences)

    # Store the result for this sample size
    kl_divergence_results[sample_size] = average_kl_divergence

    # Print confirmation for each sample size
    print(f"Average K-L Divergence for {sample_size} sparse samples: {average_kl_divergence:.4f}")

# Once all sample sizes are processed, save the results to a CSV file
save_kl_divergences_to_file(kl_divergence_results)

# Print completion message
print("\nK-L divergence calculations complete and saved to 'K-L Divergence sparse NN 1_10.csv'.")

Average K-L Divergence for 500 sparse samples: 0.2561
Average K-L Divergence for 1000 sparse samples: 0.3704
Average K-L Divergence for 1500 sparse samples: 0.3235
Average K-L Divergence for 2000 sparse samples: 0.4161
Average K-L Divergence for 2500 sparse samples: 0.3134
Average K-L Divergence for 3000 sparse samples: 0.3122
Average K-L Divergence for 3500 sparse samples: 0.3196
Average K-L Divergence for 4000 sparse samples: 0.3261
Average K-L Divergence for 4500 sparse samples: 0.4116
Average K-L Divergence for 5000 sparse samples: 0.3940
Average K-L Divergence for 5500 sparse samples: 0.3374
Average K-L Divergence for 6000 sparse samples: 0.3627
Average K-L Divergence for 6500 sparse samples: 0.3223
Average K-L Divergence for 7000 sparse samples: 0.4758
Average K-L Divergence for 7500 sparse samples: 0.5426
Average K-L Divergence for 8000 sparse samples: 0.3905
Average K-L Divergence for 8500 sparse samples: 0.3577
Average K-L Divergence for 9000 sparse samples: 0.2985
Average K-L

# ------------------------------------------------------------------------------------------------------------